In [2]:
# session.write_pandas(df,table_name="HR_CHURN",auto_create_table=True).show()
!pip install fosforml

     |████████████████████████████████| 40kB 4.4MB/s eta 0:00:01
     |████████████████████████████████| 1.9MB 22.3MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 102.3MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 114kB/s s eta 0:00:01��█████████████████             | 118.6MB 106.8MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 102.0MB/s eta 0:00:01
     |████████████████████████████████| 81kB 15.1MB/s eta 0:00:01
     |████████████████████████████████| 61kB 15.7MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 82.0MB/s eta 0:00:01
     |████████████████████████████████| 51kB 13.3MB/s eta 0:00:01
     |████████████████████████████████| 133kB 107.8MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 70.9MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 57.1MB/s eta 0:00:01
     |████████████████████████████████| 98.7MB 183kB/s  eta 0:00:01��██▏                | 46.8MB 82.1MB/s eta 0:00:01    |███████

In [1]:
from fosforml.model_manager.snowflakesession import get_session

ModuleNotFoundError: No module named 'fosforml'

In [5]:
session = get_session()
session.get_current_schema().replace('"','')


'TEST_DUMMY_SCHEMA_1'

In [6]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

In [7]:
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [23]:
import pandas as pd
df = pd.read_csv("HR_CHURN.csv")
# snow_df = session.table("HR_CHURN")
snow_df = session.create_dataframe(df)
snow_df = snow_df.drop('Unnamed: 0')
snow_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"DEPARTMENT"  |"SALARY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.38                  |0.53               |2                 |157                     |3                     |0                |1       |0                        |sales         |low       |
|0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |sales         |medium    |
|0.11                  |0.88               |7

In [24]:
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])
input_data_frame.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"DEPARTMENT"  |"SALARY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.72                  |0.87               |5                 |223                     |5                     |0                |1       |0                        |sales         |low       |
|0.37                  |0.52               |2                 |159                     |3                     |0                |1       |0                        |sales         |low       |
|0.41                  |0.5                |2

In [25]:
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']
input_data_frame.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.72                  |0.87               |5                 |223                     |5                     |0                |1       |0                        |
|1.0       |7.0           |0.37                  |0.52               |2                 |159                     |3                     |0                |1       |0                        |
|1.0       |7.0           |0.41              

In [30]:
from snowflake.ml.modeling.pipeline import Pipeline

In [35]:
# model = XGBClassifier(input_cols=feature_cols, label_cols=target_col)

In [42]:
pipeline = Pipeline(
    steps = [
        ("SnowflakePiplineModel",
         XGBClassifier(input_cols=feature_cols, label_cols=target_col))
    ]
)

In [43]:
pipeline.fit(input_data_frame)

In [44]:
pred_snow_df = pipeline.predict(apply_label_encoding(test_df))

In [45]:
pred_snow_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"OUTPUT_SALARY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.38                  |0.53               |2                 |157                     |3                     |0                |1       |0                        |1.0              |
|2.0       |7.0           |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |

In [46]:
from fosforml import register_model

register_model(
  model_obj=pipeline,
  session=session,
  name="Testmode02092024",
  snowflake_df=pred_snow_df,
  dataset_name="HR_CHURN",
  dataset_source="Dataset",
  source="Notebook",
  description="This is a Snowflake model",
  flavour="snowflake",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_8D3FB8A6_A886_4236_9B35_73EB1303C4BF_FDC_TESTMODE02092024' registered successfully."

In [54]:
model_obj = model_registry.get_model(model_name="TestModel")

In [55]:
model = model_obj.version(version_name="v1")

In [58]:
inptu_df ,test_df = session.table("HR_CHURN").randomSplit([0.75, 0.25])

In [59]:
test_df = apply_label_encoding(test_df)

In [75]:
test_df_pandas = test_df.to_pandas()

In [76]:
session.write_pandas(test_df_pandas,table_name="TEST_DATA_HR",overwrite=True).show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |7.0           |13            |0.41                  |0.55               |2                 |148                     |3                     |0                |1     

In [80]:
test_df_fetch = session.table("TEST_DATA_HR")

In [81]:
test_df_fetch.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |7.0           |13            |0.41                  |0.55               |2                 |148                     |3                     |0                |1     

In [82]:
model.run(test_df_fetch, function_name="predict").to_pandas()

,SALARY,DEPARTMENT,Unnamed: 0,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,OUTPUT_SALARY
0,1.0,3.0,34,0.84,0.87,4,246,6,0,1,0,1.0
1,2.0,6.0,66,0.43,0.54,2,153,3,0,1,0,2.0
2,1.0,7.0,94,0.09,0.83,6,255,4,0,1,0,1.0
3,1.0,8.0,130,0.10,0.83,6,292,4,0,1,0,1.0
4,1.0,7.0,163,0.79,0.89,5,239,5,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7447,2.0,5.0,14821,0.09,0.95,7,256,4,0,1,0,2.0
7448,2.0,9.0,14859,0.45,0.54,2,129,3,0,1,0,2.0
7449,1.0,7.0,14888,0.39,0.49,2,142,3,0,1,0,1.0
7450,2.0,7.0,14945,0.14,0.75,4,277,5,1,1,0,1.0


In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_model",
  description="This is a test model",
  flavour="snowflake",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)